In [1]:
import cv2
import torch
import os
import psutil
from ultralytics import YOLO
import time
import numpy as np
import requests

In [2]:
# Create a Dataset folder if it doesn't exist
if not os.path.exists('Dataset'):
    os.makedirs('Dataset')

In [3]:
# Detect hardware capabilities
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpu_available = torch.cuda.is_available()
total_ram = psutil.virtual_memory().total / (1024 ** 3)  # Convert bytes to GB
cpu_cores = psutil.cpu_count(logical=False)  # Physical cores
power_mode = "HIGH" if gpu_available and total_ram > 8 else "MEDIUM" if cpu_cores > 4 else "LOW"

print(f"Device: {device}, RAM: {total_ram:.2f}GB, CPU Cores: {cpu_cores}, Power Mode: {power_mode}")

Device: cpu, RAM: 15.35GB, CPU Cores: 8, Power Mode: MEDIUM


In [5]:
# Select appropriate YOLO model based on power mode
if power_mode == "HIGH":
    model_path = "yolov8m.pt"  # High model for high-end GPUs
    frame_skip = 1  # Process every frame
    conf_threshold = 0.25
    fps_limit = 60
elif power_mode == "MEDIUM":
    model_path = "yolov8s.pt"  # Small model for mid-range devices
    frame_skip = 2  # Process every 2nd frame
    conf_threshold = 0.3
    fps_limit = 45
else:
    model_path = "yolov8n.pt"  # Nano model for low-power devices
    frame_skip = 3  # Process every 3rd frame
    conf_threshold = 0.35
    fps_limit = 20

In [6]:
tracking_threshold = 100  # Distance threshold for the same person
histogram_threshold = 0.6  # Minimum histogram correlation to be considered the same person
time_threshold = 5 

In [7]:
# Load YOLO model and send to device
model = YOLO(model_path)
model.to(device)

100%|██████████| 21.5M/21.5M [02:07<00:00, 177kB/s] 


In [8]:
# Open video stream (replace with drone RTSP URL if needed)
video_source = 0  # Use drone RTSP stream if applicable
cap = cv2.VideoCapture(video_source)

# Set FPS limit dynamically for power efficiency
cap.set(cv2.CAP_PROP_FPS, fps_limit)

True

In [9]:
# Initialize tracking structures
persons_tracker = []
tracked_persons = {}
frame_count = 0
face_id_counter = 0

In [10]:
def euclidean_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame, exiting...")
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Skip frames dynamically to save power

    # Flip the frame horizontally for mirroring effect
    frame = cv2.flip(frame, 1)

    # Resize frame to 640x480 for faster processing (comment if full resolution is needed)
    frame = cv2.resize(frame, (640, 480))

    # Run YOLO inference with streaming and lower confidence threshold
    results = model(frame, verbose=False, stream=True, conf=conf_threshold)

    # Process detections and draw bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = box.conf[0].item()  # Confidence score
            cls = int(box.cls[0].item())  # Class index

            # Only process "person" class (class 0 for most YOLO models)
            if model.names[cls] == "person" and conf >= conf_threshold:
                # Compute the center of the bounding box
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                new_center = (center_x, center_y)

                # Check if this person is already tracked
                is_new_person = True
                for existing_person in persons_tracker:
                    # If the distance between centers is less than a threshold, it's the same person
                    if euclidean_distance(new_center, existing_person) < 100:
                        is_new_person = False
                        break

                if is_new_person:
                    # If it's a new person, save the image and add it to the tracker
                    cropped_image = frame[y1:y2, x1:x2]
                    timestamp = time.strftime("%Y%m%d-%H%M%S")
                    file_name = f"Dataset/person_{timestamp}_{face_id_counter}.jpg"
                    cv2.imwrite(file_name, cropped_image)  # Save the image

                    # Add the new person's center to the tracker
                    persons_tracker.append(new_center)
                    face_id_counter += 1
                    print(f"Saved detected person image to: {file_name}")

            # Skip drawing bounding boxes for non-person objects
            if model.names[cls] != "person":
                continue

            # Draw bounding box for detected person
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Adaptive Drone Detection (Mirrored)", frame)

    # Print the count of unique persons detected
    print(f"Unique persons detected so far: {len(persons_tracker)}")

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Ultralytics YOLOv8.0.43  Python-3.10.11 torch-2.2.0+cpu CPU
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs


Saved detected person image to: Dataset/person_20250219-215605_0.jpg
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique persons detected so far: 1
Unique person